In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 


In [2]:
# Importing Libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## READING DATA SET

In [3]:
data = pd.read_csv('Dataset_Uber Traffic.csv')
# Check the head of the dataset
data.head()

,DateTime,Junction,Vehicles,ID
0,01/11/15 0:00,1,15,20151101001
1,01/11/15 1:00,1,13,20151101011
2,01/11/15 2:00,1,10,20151101021
3,01/11/15 3:00,1,7,20151101031
4,01/11/15 4:00,1,9,20151101041


Inspect the various aspects of the Uber dataframe

In [4]:
data.shape

(48120, 4)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  48120 non-null  object
 1   Junction  48120 non-null  int64 
 2   Vehicles  48120 non-null  int64 
 3   ID        48120 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [6]:
data.describe()

,Junction,Vehicles,ID
count,48120.000000,48120.000000,4.812000e+04
mean,2.180549,22.791334,2.016330e+10
std,0.966955,20.750063,5.944854e+06
min,1.000000,1.000000,2.015110e+10
25%,1.000000,9.000000,2.016042e+10
50%,2.000000,15.000000,2.016093e+10
75%,3.000000,29.000000,2.017023e+10
max,4.000000,180.000000,2.017063e+10


Let's do a sanity check on the dataframe for missing values.

In [7]:
# percentage of missing values in each column
round(data.isnull().sum()/len(data.index), 2)*100

DateTime    0.0
Junction    0.0
Vehicles    0.0
ID          0.0
dtype: float64

In [8]:
# Loking for duplicate values,if any
data.duplicated().sum()

0

In the above 

from datetime import datetime
data['DateTime'] = pd.to_datetime(data['DateTime'])

data['Date'] = data['DateTime'].dt.date
data['Time'] = data['DateTime'].dt.time

data = data.drop(columns=['DateTime'])


,DateTime,Junction,Vehicles,ID
0,01/11/15 0:00,1,15,20151101001
1,01/11/15 1:00,1,13,20151101011
2,01/11/15 2:00,1,10,20151101021
3,01/11/15 3:00,1,7,20151101031
4,01/11/15 4:00,1,9,20151101041
...,...,...,...,...
48115,30/06/17 19:00,4,11,20170630194
48116,30/06/17 20:00,4,30,20170630204
48117,30/06/17 21:00,4,16,20170630214
48118,30/06/17 22:00,4,22,20170630224


In [16]:
data["DateTime"] = pd.to_datetime(data["DateTime"])
traffic_summary = (
    data.groupby(["Junction", pd.Grouper(key="DateTime", freq="H")])["Vehicles"]
    .sum()
    .reset_index()
)

In [18]:
traffic_summary.head()

,Junction,DateTime,Vehicles
0,1,2015-01-11 00:00:00,15
1,1,2015-01-11 01:00:00,13
2,1,2015-01-11 02:00:00,10
3,1,2015-01-11 03:00:00,7
4,1,2015-01-11 04:00:00,9
